In [405]:
%%capture
%run 03_block.ipynb

We can validate a mined block pow very easily. 

In [406]:
def val_pow(mb):
    mb_b        = bytes(mb)
    candidate   = mb_b + str(mb.nonce).encode()
    candidate_h = sha(candidate)
    if candidate_h[:mb.diff] == '0'*mb.diff: return True
    else                                   : return False
    
assert val_pow(mb)

If we change something in the block the pow should break.

In [407]:
mb.prev_hash = rh()
assert not val_pow(mb)

Account state is a simple object that holds the current balance and nonce of an account.

In [408]:
class AccountState:
    def __init__(self): self.balance,self.nonce = 0,0
    def update(self, balance, nonce): self.balance,self.nonce = balance,nonce
    def update_nonce(self,): self.nonce += 1
    def add_balance(self, value): self.balance += value
    def sub_balance(self, value): self.balance -= value
    def __str__(self): return f'balance: {self.balance}\nnonce: {self.nonce}'

The state stores the dictionary `state` which stores the `AccountState` for each address.

In [409]:
class State:
    def __init__(self): self.state={}
    def update(self, pub, balance, nonce): self.state[pub].update(balance, nonce)
    def new   (self, pub): self.state[pub] = AccountState()
    def is_new(self, pub): return pub not in self.state
    
    def init(self, genesis):
        assert genesis.number == 0
        assert genesis.prev_hash == '0x0'
        for tx in genesis.txs:
            miner = genesis.miner
            if self.is_new(tx.fr): self.new(tx.fr)
            if self.is_new(tx.to): self.new(tx.to)
            if self.is_new(miner): self.new(miner)
            assert val_sig(tx)
            self.state[tx.fr].update_nonce()
            self.state[tx.to].add_balance(tx.value)
        
    def apply(self, tx, miner):
        if self.is_new(tx.fr): self.new(tx.fr)
        if self.is_new(tx.to): self.new(tx.to)
        if self.is_new(miner): self.new(miner)
        assert val_sig(tx)
        assert self.state[tx.fr].nonce  == tx.nonce
        assert self.state[tx.fr].balance - tx.value > 0
        self.state[tx.fr].update_nonce()
        self.state[tx.fr].sub_balance(tx.value)
        self.state[tx.to].add_balance(tx.value)
        self.state[miner].add_balance(tx.fee)
        return True
            
    def __str__(self):
        return '\n'.join(f'{ph(k)}\n{v}\n' for k,v in self.state.items())

## Blockchain

Consists of a number of blocks. Each block points to the block that came before it. Imagine a linked list of blocks linked by its `prev_hash`.

In [434]:
class Blockchain:
    def __init__(self, genesis):
        self.state  = State()
        self.blocks = []
        self.state.init(genesis)
        self.blocks.append(genesis)

    def candidate(self, txs):
        return Block(txs, self.blocks[-1].hash, len(self.blocks))
        
    def add(self, mb):
        assert self.val(mb)
        for tx in mb.txs: assert self.state.apply(tx, mb.miner)
        self.blocks.append(mb)
        return True
    
    def val(self, mb): 
        assert val_pow(mb)
        assert mb.number == len(self.blocks)
        if len(self.blocks) > 0: assert self.blocks[-1].hash == mb.prev_hash
        return True
    
    def __str__(self): return '\n'.join(f'{str(block)}\n----\n' for block in self.blocks)

In [435]:
acc1,acc2,acc3 = Account(),Account(),Account()
miner = Miner(acc3)

In [436]:
tx1 = acc1.signed_tx(acc2, 24, 0.1)
tx2 = acc2.signed_tx(acc1, 18, 0.12)

gb = miner.mine_genesis([tx1, tx2], reward=200)

In [437]:
bc = Blockchain(gb); print(bc)

time:		Sat Mar 27 21:38:32 2021
number:		0
prev_hash:	0x0

txs:		

time:	Sat Mar 27 21:38:32 2021
from:	💀 0x1a430D072f4C47...102
to:	💀 0x1a430D072f4C47...102
value:	200 ether
fee:	0 ether
nonce:	0
hash:	🔿 d964b731c5b756fa...b51
signed:	true

time:	Sat Mar 27 21:38:32 2021
from:	🔒 0xAcdEE2e016E9ff...5C6
to:	👩 0x03B305027c6fDE...94a
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	🔴 ce8152c6b9994b32...425
signed:	true

time:	Sat Mar 27 21:38:32 2021
from:	👩 0x03B305027c6fDE...94a
to:	🔒 0xAcdEE2e016E9ff...5C6
value:	18 ether
fee:	0.12 ether
nonce:	0
hash:	🕖 f017ff12b1631626...263
signed:	true

miner:		💀 0x1a430D072f4C47...102
hash:		📵 8f31d00fb2d89caa...d8d
diff:		1
reward:		200
nonce:		11
----



In [438]:
tx1 = acc1.signed_tx(acc2, 3, 0.05)
tx2 = acc2.signed_tx(acc1, 9, 0.16)

block = bc.candidate([tx1,tx2])
mb    = miner.mine(block, 1, 100)
bc.add(mb); print(bc)

time:		Sat Mar 27 21:38:32 2021
number:		0
prev_hash:	0x0

txs:		

time:	Sat Mar 27 21:38:32 2021
from:	💀 0x1a430D072f4C47...102
to:	💀 0x1a430D072f4C47...102
value:	200 ether
fee:	0 ether
nonce:	0
hash:	🔿 d964b731c5b756fa...b51
signed:	true

time:	Sat Mar 27 21:38:32 2021
from:	🔒 0xAcdEE2e016E9ff...5C6
to:	👩 0x03B305027c6fDE...94a
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	🔴 ce8152c6b9994b32...425
signed:	true

time:	Sat Mar 27 21:38:32 2021
from:	👩 0x03B305027c6fDE...94a
to:	🔒 0xAcdEE2e016E9ff...5C6
value:	18 ether
fee:	0.12 ether
nonce:	0
hash:	🕖 f017ff12b1631626...263
signed:	true

miner:		💀 0x1a430D072f4C47...102
hash:		📵 8f31d00fb2d89caa...d8d
diff:		1
reward:		200
nonce:		11
----

time:		Sat Mar 27 21:38:34 2021
number:		1
prev_hash:	📵 8f31d00fb2d89caa...d8d

txs:		

time:	Sat Mar 27 21:38:34 2021
from:	💀 0x1a430D072f4C47...102
to:	💀 0x1a430D072f4C47...102
value:	100 ether
fee:	0 ether
nonce:	1
hash:	💑 2bf6db11c9466b4a...836
signed:	true

time:	Sat Mar 27 21:38:34 2021
from:	🔒

In [439]:
acc4 = Account()
tx1 = acc2.signed_tx(acc4, 12, 1.5)

block = bc.candidate([tx1])
mb    = miner.mine(block, 2, 100)
bc.add(mb); print(bc)

time:		Sat Mar 27 21:38:32 2021
number:		0
prev_hash:	0x0

txs:		

time:	Sat Mar 27 21:38:32 2021
from:	💀 0x1a430D072f4C47...102
to:	💀 0x1a430D072f4C47...102
value:	200 ether
fee:	0 ether
nonce:	0
hash:	🔿 d964b731c5b756fa...b51
signed:	true

time:	Sat Mar 27 21:38:32 2021
from:	🔒 0xAcdEE2e016E9ff...5C6
to:	👩 0x03B305027c6fDE...94a
value:	24 ether
fee:	0.1 ether
nonce:	0
hash:	🔴 ce8152c6b9994b32...425
signed:	true

time:	Sat Mar 27 21:38:32 2021
from:	👩 0x03B305027c6fDE...94a
to:	🔒 0xAcdEE2e016E9ff...5C6
value:	18 ether
fee:	0.12 ether
nonce:	0
hash:	🕖 f017ff12b1631626...263
signed:	true

miner:		💀 0x1a430D072f4C47...102
hash:		📵 8f31d00fb2d89caa...d8d
diff:		1
reward:		200
nonce:		11
----

time:		Sat Mar 27 21:38:34 2021
number:		1
prev_hash:	📵 8f31d00fb2d89caa...d8d

txs:		

time:	Sat Mar 27 21:38:34 2021
from:	💀 0x1a430D072f4C47...102
to:	💀 0x1a430D072f4C47...102
value:	100 ether
fee:	0 ether
nonce:	1
hash:	💑 2bf6db11c9466b4a...836
signed:	true

time:	Sat Mar 27 21:38:34 2021
from:	🔒